# Практическое занятие №2 - локализация объектов на изображениях

1. Разметка данных с помощью виджета QSL для Jupyter (https://github.com/faustomorales/qsl)
2. Знакомство с архитектурой нейронных сетей для локализации объектов на изображении.
3. Разработка простой системы управления по данным с видео-камеры.

In [ ]:
!pip install -U -q qsl

     |████████████████████████████████| 1.8 MB 8.7 MB/s 
     |████████████████████████████████| 1.6 MB 47.4 MB/s 


In [ ]:
# These next two lines are only required on Google Colab.
# They are not needed in Jupyter Lab or Jupyter Notebook.
import google.colab
google.colab.output.enable_custom_widget_manager()

In [ ]:
# Widgets have been tested on JupyterLab >= 3 and
# Jupyter Notebook >= 6
import qsl
import qsl.testing.files

In [ ]:
labeler = qsl.MediaLabeler(
    items=[
        # You can pass a local path, a URL, or an S3 path.
        {"target": qsl.testing.files.TEST_IMAGES[0]},
        {"target": "https://picsum.photos/id/237/200/300"},
        # You can label videos!
        {"target": "https://test-videos.co.uk/vids/bigbuckbunny/mp4/h264/360/Big_Buck_Bunny_360_10s_2MB.mp4", "type": "video"},
        # Optionally set initial default labels for images. Note that labels are always provided
        # as a list of strings whether they are multiselect / single select or freeform.
        {"target": "https://picsum.photos/id/1025/500/500", "defaults": {"image": {"Type": ["Dog"]}}},
        # Provide arbitrary string metadata and it will appear just below the image in the widget.
        {"target": "https://picsum.photos/id/1062/500/500", "metadata": {"source": "picsum"}},
        # Set labels in advance. These images will be considered "complete" for progress calculation purposes.
        {"target": "https://picsum.photos/id/1074/500/500", "labels": { "image": {"Type": ["Cat"]}}},
        {"target": "https://picsum.photos/id/219/500/500"}
    ],
    config={
        "image": [
            {"name": "Type", "options": [
                {"name": "Dog", "shortcut": "d"},
                {"name": "Cat", "shortcut": "c"},
                {"name": "Neither", "shortcut": "n"}
            ]},
            {"name": "Tags", "options": [], "freeform": True, "multiple": True},
            {"name": "Comment", "freeform": True}
        ],
        "regions": [
            {"name": "Part", "options": [{"name": "Nose", "shortcut": "n"}, {"name": "Eye", "shortcut": "e"}]}
        ]
    }
)
labeler.mode = "dark"
labeler.maxViewHeight = 256
labeler

MediaLabeler(buttons={'prev': False, 'next': True, 'save': True, 'config': True, 'delete': False, 'ignore': Tr…

In [ ]:
# You can access the current state of the image labels at any time.
# Mostly self-explanatory. The segmentation maps provide map
# dimensions (width/height) and a run-length encoded representation
# with classes i (inside mask) and o (outside of mask). The mask
# may be lower-res than the image (you can tell from the dimensions)
# if the image was large, so you'll have to upsample and interpolate
# the map up to true image size. This little twist makes the in-browser
# behavior more memory efficient at the cost of mask precision.
labeler.items

[{'target': '/usr/local/lib/python3.7/dist-packages/qsl/testing/data/image1.jpg'},
 {'target': 'https://picsum.photos/id/237/200/300'},
 {'target': 'https://test-videos.co.uk/vids/bigbuckbunny/mp4/h264/360/Big_Buck_Bunny_360_10s_2MB.mp4',
  'type': 'video',
  'labels': [{'labels': {'image': {},
     'polygons': [{'points': [{'x': 0.26371221428847796,
         'y': 0.6212221662715348},
        {'x': 0.2285553327531164, 'y': 0.7540389253462455},
        {'x': 0.353801723222842, 'y': 0.8126345543497944},
        {'x': 0.3779720792784031, 'y': 0.5430946609334697},
        {'x': 0.26371221428847796, 'y': 0.6212221662715348}],
       'labels': {'Part': ['Eye']}}],
     'masks': [],
     'dimensions': {'width': 640, 'height': 360},
     'boxes': []},
    'timestamp': 2.87},
   {'labels': {'image': {},
     'polygons': [],
     'masks': [],
     'dimensions': {'width': 640, 'height': 360},
     'boxes': [{'pt1': {'x': 0.4175235710056848, 'y': 0.5352819103996631},
       'labels': {},
       'p